In [ ]:
import torch
from torch import nn

import random
from operator import sub

In [ ]:
device = ""
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

# TODO: the parameters of the trained network should be saved into a file.
class c_function():
    def __init__(self, isBoolean):
        self.isBoolean = isBoolean
        self.hidden_layer_size = -1
    
    def __call__(self, trial_code):
        return self.network(torch.FloatTensor(trial_code).to(device)).detach().numpy()
    
    def train(self, raw_train_data):
        
        if self.hidden_layer_size <= 0:
            self.hidden_layer_size = 4
        
        while True:
            print (f'Started training. {self.hidden_layer_size = }')
            
            train_loss, val_loss = self.try_train(self.hidden_layer_size, raw_train_data)
            print(f'{self.hidden_layer_size = } | {train_loss = :.2f} {val_loss = :.2f}')

            if (train_loss < 0.5) and (val_loss - train_loss > 0.2):
                print ('Please provide more training data')
                break                
            if train_loss >= 0.4:
                self.hidden_layer_size = self.hidden_layer_size + 4
                print ('Complexity increased.')
                continue
            elif (0.1 <= train_loss) and (train_loss < 0.4) and (val_loss < 0.4):  # This condition is under question
                self.hidden_layer_size = self.hidden_layer_size + 4
                print ('Complexity increased.')
                continue
            elif (train_loss < 0.1) and (val_loss >= 0.1):
                print ('Please provide more training data')
                break

            elif (train_loss < 0.1) and (val_loss < 0.1):
                print ('Ready')
                break
            else:
                raise Exception('Unexpected situation')
                
    
    def try_train(self, hidden_layer_size, raw_train_data, print_intermediate=False):
        learning_rate = 0.005
        epochs = 1500
        
        data_inputs = torch.FloatTensor([data_sample[0] for data_sample in raw_train_data]).to(device)
        data_outputs = torch.FloatTensor([data_sample[1] for data_sample in raw_train_data]).to(device)
        dataset = torch.utils.data.TensorDataset(data_inputs, data_outputs)
    
        self.network = nn.Sequential(
            nn.Linear(data_inputs.size()[-1], hidden_layer_size),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_layer_size, hidden_layer_size),
            nn.LeakyReLU(0.2),
#             nn.Linear(hidden_layer_size, hidden_layer_size),
#             nn.LeakyReLU(0.2),
            nn.Linear(hidden_layer_size, data_outputs.size()[-1])
        ).to(device)
        if self.isBoolean:
            self.network = nn.Sequential(self.network, nn.Sigmoid())
        
        optimizer = torch.optim.Adam(self.network.parameters(), lr=learning_rate)
        
        
        criterion = nn.BCELoss() if self.isBoolean else nn.L1Loss()
        
        # TODO: need to use cross-validation
        train_size = int(0.7 * len(dataset))
        val_size = len(dataset) - train_size
    
        train_set, val_set = torch.utils.data.random_split(dataset, [train_size, val_size])
        train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
        val_loader = torch.utils.data.DataLoader(val_set, batch_size=val_size, shuffle=True)
        
        for epoch in range(epochs):
            for n, samples in enumerate(train_loader):

                inputs = samples[0]
                expected_outputs = samples[1]

                optimizer.zero_grad()
                
                actual_outputs = self.network(inputs)                
        
                train_loss = criterion(actual_outputs, expected_outputs)
                
                train_loss.backward()
                optimizer.step()
                                
            for n, samples in enumerate(val_loader):

                inputs = samples[0]
                expected_outputs = samples[1]

                actual_outputs = self.network(inputs)
                val_loss = criterion(actual_outputs, expected_outputs)
               
            if print_intermediate:
                print(f'E {epoch} | {train_loss = :.2f} {val_loss = :.2f}')
            
        return train_loss, val_loss

In [ ]:
# Utility for creation of a training data for errors detection

def get_errors_test_data(n):
    res = []
    for _ in range(n):

        code = [random.randint(0, 9) for x in range(4)]
        has_errors = 1 if len(code) != len(set(code)) else 0
        res.append([code, [has_errors]])
        
    return res

print(get_errors_test_data(1000))

In [ ]:
# Utility for creation of a training data for bulls count

def generate_list_without_duplicates(possible_numbers, length):
    return random.sample(possible_numbers, length)

def get_bulls_test_data(n):
    res = []
    for _ in range(n):

        # I cannot generate secret_code and guessed_code at first, and then calculate the 
        # bulls count. It's much easier, but it provides a biased data with a lot of 
        # samples with 0 bulls and very few samples with 3 and 4 bulls.
        
        secret_code = generate_list_without_duplicates(range(10), 4)
        bulls_count = random.randint(0, 4)
        
        unused_numbers = set(range(10)) - set(secret_code)
        
        guessed_code = generate_list_without_duplicates(unused_numbers, 4)
        
#         indices_to_place_bulls_to = generate_list_without_duplicates(range(4), bulls_count)
#         bulls = generate_list_without_duplicates(secret_code, bulls_count)
        bulls_indices = generate_list_without_duplicates(range(4), bulls_count)
                
        for bull_index in bulls_indices:
            guessed_code[bull_index] = secret_code[bull_index]
        
        res.append([secret_code + guessed_code, [bulls_count]])
        
    return res

get_bulls_test_data(10)
# print(get_bulls_test_data(10))

In [ ]:
# Create dependencies
has_errors = c_function(True)

In [ ]:
find_bulls_count = c_function(False)

In [ ]:
# Train dependencies
has_errors.train(get_errors_test_data(32*160))

In [ ]:
# Train dependencies
find_bulls_count.train(get_bulls_test_data(32*40))

In [ ]:
def try_guess(trial_code):    
    code_has_errors = has_errors(trial_code)
    print(code_has_errors)
    
    if(round(code_has_errors[0]) == 1):
        print('error')
    else:
        print(f'bulls_count: {round(find_bulls_count(trial_code + g_secretCode)[0])}')
        #print(f'{result[1]} bulls; {result[2]} cows.')

In [ ]:
g_secretCode = [5, 4, 3, 2]

In [ ]:
try_guess(     [2, 5, 4, 3])